# __creating and cleaning dataset - definitions.csv__
1. [creating dataset - definitions.csv](#creating-definitions.csv)
2. [drop words without definitions and definitions without words](#drop-words-without-definitions-and-definitions-without-words)
3. [remove special characters](#remove-special-characters)
    * [retrieve special characters](#retrieve-special-characters)
    * [delete special characters](#delete-special-characters)
4. [extract words without definition from dataset and save to missing-words.csv](#extract-words-from-definitions-which-have-no-definition-in-dataset-and-save-to-missing-words.csv)

### creating definitions.csv
---

In [2]:
import pandas as pd
from lib.connect_db import get_engine
from lib.print_error import print_error
from nltk.corpus import stopwords

In [ ]:
engine, meta = get_engine()

df = pd.read_sql("definitions", engine)
df.to_csv("db/definitions.csv", sep=":", columns = ["word", "definition"], header = False, index=False, mode = "w")

### drop words without definitions and definitions without words
---

In [ ]:
df = pd.read_csv("db/definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
words = pd.DataFrame(columns=["word"], index=None)

ww = pd.isnull(df['word'])

null_word_indices = []
for id,w in enumerate(ww):
    if w is True:
        null_word_indices.append(id)

dd = pd.isnull(df['definition'])
null_def_indices = []
for id,d in enumerate(dd):
    if d is True:
        null_def_indices.append(id)

for el in null_word_indices:
    if el in null_def_indices:
        null_def_indices.remove(el)

print("df info before deletion - ")
print(df.info())
print("\n")

df = df.drop(null_word_indices, inplace=False)
df = df.drop(null_def_indices, inplace=False)
df = df.reset_index(drop=True)

print("df info after deletion - ")
print("*"*10)
print(df.info())

df.to_csv("db/definitions.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="w")


### remove special characters
---
this section is divided into two parts -
* retrieve special characters
* delete special characters

#### retrieve special characters

In [ ]:
df = pd.read_csv("db//definitions.csv", names=['word','definition'], sep=":", index_col=None, keep_default_na=False, na_values=[""])

special_chars = []


def get_special_chars(defi):
    for d in list(defi):
        if (ord(d) < 48 and ord(d) != 32) or (ord(d) > 57 and ord(d)<65) or (ord(d) >90 and ord(d) < 97) or ord(d) > 122:
            if d not in special_chars:
                special_chars.append(d)
    return

df["definition"].apply(get_special_chars)

special_chars.remove(";")

print(special_chars)


#### delete special characters

In [ ]:
df = pd.read_csv("db/definitions.csv", names=['word','definition'], sep=":", index_col=None, keep_default_na=False, na_values=[""])

def remove_special_chars(defi):
    for s in special_chars:
        defi = " ".join(defi.split(s))
    index = 0
    defi = list(defi)
    while index < len(defi)-1:
        if defi[index] == " " and defi[index+1] == " ":
            del defi[index]
        else:
            index += 1
    return "".join(defi)


df["definition"] = df["definition"].apply(remove_special_chars)
df.to_csv("db/definitions.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="w")

### split definitions based on ';'
---

In [ ]:
df = pd.read_csv("db/definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
final_df = pd.DataFrame(columns=["word","definition"], index=None)

print(df.info())
print("*"*20)

drop_indices = []

def split_defs(row):
    global final_df, df
    
    index = row.name
    definition = row['definition']
    definition_list = list(definition)
    
    if ";" in definition_list:
        drop_indices.append(index)
        new_definitions = definition.split(";")
        new_definitions = [d.strip() for d in new_definitions]
        # index = df.loc[df["definition"] == definition]["word"].index
        # index = index[0]
        for dd in new_definitions:
            final_df = final_df.append({"word":df.word[index], "definition":dd}, ignore_index=True)

            
df.apply(split_defs,axis=1)

print("\n")
print("total words splitted: ", str(len(drop_indices)))
print("\n")

df = df.drop(drop_indices)
df = df.reset_index()


print("*"*20)
print("resultant df - ")
print(df.info())

df.to_csv("db/definitions.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="w")

final_df.to_csv("db/split-definitions.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="w")
print("*"*20)
print("Done")

#### **concat definitons.csv and split-definitions.csv**

In [ ]:
def concat_dfs(dfs):
    for df in dfs:
        print("\n")
        print(split_df.info())
        print("*"*20)

    print("\nconcatenating dfs..")
    return pd.concat(dfs)

In [ ]:
split_df = pd.read_csv("db/split-definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df = pd.read_csv("db/definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])

resultant_df = concat_dfs([split_df, df])
resultant_df.to_csv("db/definitions.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="w")

#### **remove words without definitions**

the code already exists [above](#drop-words-without-definitions-and-definitions-without-words). Re-run it to remove null values

###  extract words from column definition which have no definition in dataset; save to missing-words.csv
---

#### Tasks
- ~~create a dataset by splitting definition column and obtaining all columns~~
- remove numbers from the dataset
- ~~remove words whose meaning exists~~
- stem words

In [ ]:
df = pd.read_csv("db/definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])

print(df.info())
print("*"*20)

missing_data = []
count = 0

def missing_words(definition):
    for d in str(definition).split():
        if not any(df["word"] == d) and d not in missing_data:
            missing_data.append(d)
    print("missing_data contains", str(len(missing_data)), "words")
    
df["definition"].apply(missing_words)

final_data = missing_data
print("total words obtained:", len(final_data))

defi_df = pd.DataFrame({"word":final_data}, columns = ["word"], index=None)
defi_df = defi_df.drop_duplicates(keep='first')

print("\n")
print("definitions df obtained - ")
print(defi_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135949 entries, 0 to 135948
Data columns (total 2 columns):
word          135949 non-null object
definition    135949 non-null object
dtypes: object(2)
memory usage: 2.1+ MB
None
********************
missing_data contains 2 words
missing_data contains 3 words
missing_data contains 3 words
missing_data contains 5 words
missing_data contains 5 words
missing_data contains 5 words
missing_data contains 6 words
missing_data contains 6 words
missing_data contains 7 words
missing_data contains 7 words
missing_data contains 8 words
missing_data contains 9 words
missing_data contains 10 words
missing_data contains 10 words
missing_data contains 10 words
missing_data contains 10 words
missing_data contains 11 words
missing_data contains 12 words
missing_data contains 12 words
missing_data contains 12 words
missing_data contains 12 words
missing_data contains 12 words
missing_data contains 12 words
missing_data contains 12 words
missing_data conta

In [ ]:
defi_df.to_csv("db/missing-defs-temp.csv", sep=":", columns=["word"], header=False, index=False, mode="w")

#### **concat missing-defs.csv and missing-defs-temp.csv**

In [ ]:
missing_temp_df = pd.read_csv("db/missing-defs-temp.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
missing_df = pd.read_csv("db/missing-defs.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])

result_df = concat_dfs([missing_temp_df, missing_df])
print("\nresultant df after concatenating missing-defs.csv and missing-defs-temp.csv..")
print(result_df.info())
